<a href="https://colab.research.google.com/github/NjokiM/UON_CS/blob/main/Testing_Classification_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#connecting to google drive 
from google.colab import drive
drive.mount('/content/gdrive')
%cd 'gdrive/MyDrive'

Mounted at /content/gdrive
/content/gdrive/MyDrive


In [2]:
#import libraries
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Data
data = pd.read_csv('MAUREEN NJOKI - credit_risk_testingdata - MAUREEN NJOKI - credit_risk_testingdata.csv')
data.head()

,Unnamed: 0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,1220,22,36900,RENT,5.0,VENTURE,A,16000,7.90,1,0.43,N,2
1,6889,22,50004,MORTGAGE,4.0,DEBTCONSOLIDATION,A,6250,NaN,0,0.12,N,4
2,29945,40,28000,RENT,2.0,VENTURE,C,5000,12.99,0,0.18,Y,13
3,15469,25,138000,MORTGAGE,0.0,MEDICAL,B,5000,11.86,0,0.04,N,4
4,25138,29,80000,MORTGAGE,2.0,PERSONAL,A,5000,7.74,0,0.06,N,9


In [4]:
data.columns = data.columns.str.replace('Unnamed: 0', 'ID')

In [5]:
columns =list(data.columns)
columns

['ID',
 'person_age',
 'person_income',
 'person_home_ownership',
 'person_emp_length',
 'loan_intent',
 'loan_grade',
 'loan_amnt',
 'loan_int_rate',
 'loan_status',
 'loan_percent_income',
 'cb_person_default_on_file',
 'cb_person_cred_hist_length']

In [6]:
data.describe()

,ID,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,8146.000000,8146.000000,8.146000e+03,7908.000000,8146.000000,7352.000000,8146.000000,8146.000000,8146.000000
mean,16313.148539,27.731770,6.648822e+04,4.824608,9682.933342,11.027690,0.221704,0.170849,5.793273
std,9375.383212,6.392684,5.034207e+04,4.293455,6313.025846,3.237725,0.415418,0.106776,4.020205
min,0.000000,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,8172.750000,23.000000,3.900000e+04,2.000000,5000.000000,7.900000,0.000000,0.090000,3.000000
50%,16273.500000,26.000000,5.546500e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000
75%,24398.750000,30.000000,8.000000e+04,7.000000,12300.000000,13.470000,0.000000,0.230000,8.000000
max,32570.000000,144.000000,1.200000e+06,123.000000,35000.000000,21.740000,1.000000,0.710000,30.000000


In [7]:
# Check the column names and datatypes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8146 entries, 0 to 8145
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          8146 non-null   int64  
 1   person_age                  8146 non-null   int64  
 2   person_income               8146 non-null   int64  
 3   person_home_ownership       8146 non-null   object 
 4   person_emp_length           7908 non-null   float64
 5   loan_intent                 8146 non-null   object 
 6   loan_grade                  8146 non-null   object 
 7   loan_amnt                   8146 non-null   int64  
 8   loan_int_rate               7352 non-null   float64
 9   loan_status                 8146 non-null   int64  
 10  loan_percent_income         8146 non-null   float64
 11  cb_person_default_on_file   8146 non-null   object 
 12  cb_person_cred_hist_length  8146 non-null   int64  
dtypes: float64(3), int64(6), object(4

In [8]:
# col_type = data.dtypes
# col_type
# cat_col = ['cb_person_default_on_file', 'loan_grade', 'loan_intent', 'person_home_ownership']



In [9]:
# Import LabelEncoder
from sklearn import preprocessing
#creating labelEncoder
le = preprocessing.LabelEncoder()
data['cb_person_default_on_file']=le.fit_transform(data['cb_person_default_on_file'])

In [10]:
# Split data into train and test sets
X = data.loc[:,['ID','person_income','loan_status']]
y = data['cb_person_default_on_file']
print(X, y)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

         ID  person_income  loan_status
0      1220          36900            1
1      6889          50004            0
2     29945          28000            0
3     15469         138000            0
4     25138          80000            0
...     ...            ...          ...
8141  31176          80000            0
8142   2112          30000            0
8143  12172          74000            1
8144  10247          66000            0
8145   6381          36628            0

[8146 rows x 3 columns] 0       0
1       0
2       1
3       0
4       0
       ..
8141    0
8142    0
8143    0
8144    0
8145    0
Name: cb_person_default_on_file, Length: 8146, dtype: int64


In [11]:
#Create a Gaussian Classifier
model_GSB = GaussianNB()

# Train the model using the training sets
model_GSB.fit(X_train,y_train)

# making predictions on the testing set
y_pred = model_GSB.predict(X_test)
  
# comparing actual response values (y_test) with predicted response values (y_pred)
from sklearn import metrics
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Gaussian Naive Bayes model accuracy(in %): 82.28314238952537


In [12]:
from sklearn.ensemble import RandomForestClassifier

# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 100)  
  
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(X_train, y_train)
  
# performing predictions on the test dataset
y_pred_clf = clf.predict(X_test)
  
# metrics are used to find accuracy or error
from sklearn import metrics  
print()
  
# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred_clf))


ACCURACY OF THE MODEL:  0.7581833060556464


In [13]:
from sklearn.neighbors import KNeighborsClassifier
 
knn = KNeighborsClassifier(n_neighbors = 15)
 
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
 
# Predictions and Evaluations
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred_knn))
 
print(classification_report(y_test, y_pred_knn))

[[2010    2]
 [ 431    1]]
              precision    recall  f1-score   support

           0       0.82      1.00      0.90      2012
           1       0.33      0.00      0.00       432

    accuracy                           0.82      2444
   macro avg       0.58      0.50      0.45      2444
weighted avg       0.74      0.82      0.74      2444

